In [2]:
import pandas as pd
import os
import sys
from datetime import datetime
import yaml
import time
import requests
import urllib
import zipfile
import pprint
from tqdm import tqdm
from glob import glob
#from platforms.connect.snowpy import SnowPy

### Extract

In [10]:
print(os.getcwd())
datapath = os.getcwd() + '/nes_zips/'
#datapath = 'vol-1/NES_CPB_files/'

C:\Users\hrowe\Documents\FHWA mobility trend report\T4 - Forecasting\Year 2\modeling code\etl\NES\NES_CPB_NES_Combined


In [7]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def try_download(url, file):
    #makedir_if_needed(datapath)
    try:
        download_url(url, file)
        print(f"Downloaded {url} to {file}")
    except urllib.error.HTTPError as e:
        print(f"Couldn't find {url}, Exception: {e}")
        
def uncompress(filepath):
    # Uncompress if zip file
    if filepath[-4:].lower() == '.zip':
        zipfolder = filepath.split('/')[-1].split('.')[0]
        print(f'                Uncompressing zip file to folder {zipfolder}')
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(datapath + zipfolder)
            
            
#example url https://www2.census.gov/programs-surveys/nonemployer-statistics/datasets/2019/historical-datasets/combine19_txt.zip
#            https://www2.census.gov/programs-surveys/nonemployer-statistics/datasets/2016/combine16_txt.zip
#            https://www2.census.gov/programs-surveys/nonemployer-statistics/datasets/2012/historical-datasets/combine12_txt.zip
#            https://www2.census.gov/programs-surveys/nonemployer-statistics/datasets/2020/historical-datasets/nonemp20co.zip


In [8]:
def download_nes_county_files():
    for year in range(2000,2021):
        print(year)
        url = 'https://www2.census.gov/programs-surveys/nonemployer-statistics/datasets/' + str(year) + '/historical-datasets/nonemp' + str(year)[2:] + 'co.zip'
        file_name = 'NES_County_' + str(year) + '.zip'
        download_url(url, datapath + file_name)
        uncompress(datapath + file_name)
        os.remove(datapath + file_name)
    


In [9]:
download_nes_county_files()

2000


nonemp00co.zip: 0.00B [00:00, ?B/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hrowe\\Documents\\FHWA mobility trend report\\T4 - Forecasting\\Year 2\\modeling code\\etl\\NES\\NES_CPB_NES_Combined/nes_zips/NES_County_2000.zip'

### Transform

In [ ]:
import numpy as np

#vol-1/NES_CPB_files/NES_CPB2014/combine14.txt

#datapath = 'vol-1/NES_CPB_files/'

for year in range(2012,2020):
    print(year)
    df = pd.read_csv(datapath + 'NES_CPB' + str(year) + '/combine' + str(year)[2:] + '.txt', 
                    dtype = str)
    df_columns = df.columns
    df.columns = ['STATE', 'SDSCR', 'COUNTY', 'CTYDSCR', 'NAICS', 'NCSDSCR', 'CESTAB', 'CBP_PCT',
                 'NES_PCT', 'EST', 'EMP', 'EMP_NF', 'QP1', 'QP1_NF', 'AP', 'AP_NF', 'ESTAB', 'RCPTOT', 'RCPTOT_N_F']
    df = df[['STATE', 'SDSCR', 'COUNTY', 'CTYDSCR', 'NAICS', 'NCSDSCR', 'CESTAB', 'CBP_PCT', 
             'NES_PCT', 'EST', 'EMP', 'EMP_NF', 'QP1', 'QP1_NF', 'AP', 'AP_NF', 'ESTAB', 'RCPTOT', 'RCPTOT_N_F']]
    df.to_csv(datapath + 'NES_CPB' + str(year) + '/transformed_nonemp' + str(year)[2:] + 'co.csv', index = False)
    print(df.head)

### Load

In [ ]:
import pandas as pd
import os
import sys
from datetime import datetime
import yaml
import time
import requests
import urllib
import zipfile
import pprint
from tqdm import tqdm
from glob import glob
from platforms.connect.snowpy import SnowPy
from sqlalchemy.dialects import registry
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')


datapath = 'vol-1/NES_CPB_files/'

snowpy = SnowPy()

# login (return to add .env variabels)
account = os.environ['Account']
role = os.environ['Role']
ctx = snowpy.login(account, role, reauthenticate = True)

# Establish SnowFlake connection
warehouse = 'COMPUTE_WH'
database = 'FHWA_DEV'  # For production use FHWA_DB
schema = 'RAW'  # Source data stored in Schema RAW, transformed (curated) data stored in CUR

ctx.execute(f'USE WAREHOUSE {warehouse}')
ctx.execute(f'USE DATABASE {database}')
ctx.execute(f'USE SCHEMA {schema}') 

table_name = 'NES_CPB_COMBINED'

for year in range(2012,2020):
    print(year)
    df = pd.read_csv(datapath + 'NES_CPB' + str(year) + '/transformed_nonemp' + str(year)[2:] + 'co.csv',
                    dtype = str) 
    df['YEAR'] = year
    print(df.head)
    if year == 2012:
        snowpy.create_new_table(df = df, warehouse = warehouse, database = database, 
                            schema = schema, table = table_name, large_transfer= True, chunksize = 100000)  
    else:
        snowpy.modify_rows(df=df, warehouse=warehouse, database=database, operation = 'append', schema=schema, table=table_name, log=True, keep_alive=False)
